In [20]:
import sys
import os
analysis_dir_path = '../'
sys.path.append(analysis_dir_path)

import models
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [21]:
filters = 32
kernels = 5
units = 128

In [22]:
CNN = f'CNN_{filters}_filters_{kernels}_kernels_predictions'
CNNA = f'CNN_Attention_{filters}_filters_{kernels}_kernels_predictions'
RNN = f'RNN_{units}_units_predictions'
LSTM = f'LSTM_{units}_layers_predictions'

name_maps = {
    CNN: "CNN",
    CNNA: "CNN With Attention",
    RNN: "RNN",
    LSTM: "LSTM"
}

In [23]:
def calculate_metrics(df, rule_result_column):
    TP = ((df[rule_result_column] == 1) & (df['corr'] == 1)).sum()
    FP = ((df[rule_result_column] == 1) & (df['corr'] == 0)).sum()
    FN = ((df[rule_result_column] == 0) & (df['corr'] == 1)).sum()
    
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return f1, recall, precision

In [24]:
%%capture
import example

COMMODITY = 'nickel_no_val_20'
MODEL = CNNA
RULE_NUM = 10
confidence_levels = [0.4, 0.45, 0.5, 0.55, 0.6, 0.7, 0.8, 0.9, 0.95]

def exclude_zero(series):
    temp = series.copy()
    temp = temp[temp != 0]
    temp = temp[temp != 1]
    return temp

# A function for providing useful labels for the results
def label(results, row):
    labels = []
    max_base_model_precision = exclude_zero(results["Precision (Base Model)"]).max()
    max_base_model_recall = exclude_zero(results["Recall (Base Model)"]).max()
    max_base_model_f1 = exclude_zero(results["F1 (Base Model)"]).max()
    min_base_model_f1 = exclude_zero(results["F1 (Base Model)"]).min()

    if max_base_model_precision == row["Precision (Base Model)"]: labels.append("Best Precision")
    if max_base_model_recall == row["Recall (Base Model)"]: labels.append("Best Recall")
    if max_base_model_f1 == row["F1 (Base Model)"]: labels.append("Best F1")
    if min_base_model_f1 == row["F1 (Base Model)"]: labels.append("Worst F1")
    return ', '.join(labels)

def evaluate_df(df, properties={}):
  f1, recall, precision = calculate_metrics(df, "pred")
  prior = df["corr"].sum() / len(df)
  df.to_numpy().dump('data/test.npy')
  df = example.run_edcr()

  new_precision = df.iloc[50]["pre"]
  new_recall = df.iloc[50]["recall"]
  new_f1 = df.iloc[50]["F1"]

  percent_precision = (new_precision - precision) / precision
  percent_recall = (new_recall - recall) / recall
  percent_f1 = (new_f1 - f1) / f1

  return {
    **properties,
    "Precision (Base Model)": precision,
    "Recall (Base Model)": recall,
    "F1 (Base Model)": f1,
    "Prior": prior,
    " ": "",
    "Precision (EDCR)": df.iloc[50]["pre"],
    "Recall (EDCR)": df.iloc[50]["recall"],
    "F1 (EDCR)": df.iloc[50]["F1"],
    "  ": "",
    "Precision Improvement": df.iloc[50]["pre"] - precision,
    "Recall Improvement": df.iloc[50]["recall"] - recall,
    "F1 Improvement": df.iloc[50]["F1"] - f1
  }

for COMMODITY in [
  # 'cobalt_20'
  # 'cobalt_20', 'cobalt_no_val_20', 'cobalt_shift_20', 'cobalt_streaming_20', 'cobalt_20',
  # 'copper_20', 'copper_no_val_20', 'copper_shift_20', 'copper_streaming_20', 'copper_20',
  # 'magnesium_20', 'magnesium_no_val_20', 'magnesium_shift_20', 'magnesium_streaming_20', 'magnesium_20',
  # 'nickel_20', 'nickel_no_val_20', 'nickel_shift_20', 'nickel_streaming_20', 'nickel_20'
  'cobalt_20', 'copper_20', 'magnesium_20', 'nickel_20', 
]:
  results = []
  for MODEL in [CNN, CNNA, RNN, LSTM]:
  # for MODEL in [CNN]:
    for ALGO in ['correction', 'detection_correction']:
    # for ALGO in ['correction']:
      for RULE_NUM in [5, 10, 15, 20, 50, 100]:
      # for RULE_NUM in [5]:
        df = models.npy_to_top_n_f1_bowpy(f'../{COMMODITY}/test/predictions/test/{MODEL}.csv', f'../{COMMODITY}/test/predictions/test_F1_{ALGO}', RULE_NUM)
        results.append(evaluate_df(df, properties={"Model": MODEL, "Algorithm": ALGO, "Rule Num": RULE_NUM}))
  results = pd.DataFrame(results)
  results['Label'] = results.apply(lambda x: label(results, x), axis=1)
  results.to_excel(f'out/top_f1/{COMMODITY}_results.xlsx', index=False)

In [ ]:
%%capture
import example

COMMODITY = 'nickel_no_val_20'
MODEL = CNNA
RULE_NUM = 10
confidence_levels = [0.4, 0.45, 0.5, 0.55, 0.6, 0.7, 0.8, 0.9, 0.95]

def exclude_zero(series):
    temp = series.copy()
    temp = temp[temp != 0]
    temp = temp[temp != 1]
    return temp

# A function for providing useful labels for the results
def label(results, row):
    labels = []
    max_base_model_precision = exclude_zero(results["Precision (Base Model)"]).max()
    max_base_model_recall = exclude_zero(results["Recall (Base Model)"]).max()
    max_base_model_f1 = exclude_zero(results["F1 (Base Model)"]).max()
    min_base_model_f1 = exclude_zero(results["F1 (Base Model)"]).min()

    if max_base_model_precision == row["Precision (Base Model)"]: labels.append("Best Precision")
    if max_base_model_recall == row["Recall (Base Model)"]: labels.append("Best Recall")
    if max_base_model_f1 == row["F1 (Base Model)"]: labels.append("Best F1")
    if min_base_model_f1 == row["F1 (Base Model)"]: labels.append("Worst F1")
    return ', '.join(labels)

for COMMODITY in [
  # 'cobalt_20'
  # 'cobalt_20', 'cobalt_no_val_20', 'cobalt_shift_20', 'cobalt_streaming_20', 'cobalt_20',
  # 'copper_20', 'copper_no_val_20', 'copper_shift_20', 'copper_streaming_20', 'copper_20',
  # 'magnesium_20', 'magnesium_no_val_20', 'magnesium_shift_20', 'magnesium_streaming_20', 'magnesium_20',
  # 'nickel_20', 'nickel_no_val_20', 'nickel_shift_20', 'nickel_streaming_20', 'nickel_20'
  'cobalt_20', 'copper_20', 'magnesium_20', 'nickel_20', 
]:
  results = []
  for MODEL in [CNN, CNNA, RNN, LSTM]:
  # for MODEL in [CNN]:
    for ALGO in ['correction', 'detection_correction']:
    # for ALGO in ['correction']:
      for THRESHOLD in [0.1, 0.15, 0.2, 0.25, 0.30, 0.35, 0.4, 0.45, 0.5]:
      # for RULE_NUM in [5]:
        df = models.npy_to_threshold_f1_bowpy(f'../{COMMODITY}/test/predictions/test/{MODEL}.csv', f'../{COMMODITY}/test/predictions/test_F1_{ALGO}', THRESHOLD, exclude_models=[])
        results.append(evaluate_df(df, properties={"Model": MODEL, "Algorithm": ALGO, "Threshold": THRESHOLD}))
  results = pd.DataFrame(results)
  results['Label'] = results.apply(lambda x: label(results, x), axis=1)
  results.to_excel(f'out/threshold/{COMMODITY}_results.xlsx', index=False)

TypeError: evaluate_df() got an unexpected keyword argument 'properties'